This notebook:
1. reads in .csv files with specs and timeseries power that came from [Get_ERCOT_EIA_Data.ipynb](Get_ERCOT_EIA_Data.ipynb)
2. converts specs from EIA and ERCOT into modeling parameters
3. makes forecasts 

It is based on https://github.com/williamhobbs/PVPMC_2025/blob/main/Process_Specs_and_Data.ipynb.

For now, I'm leaving out the parameter optimization (gcr, loss factor, gamma pdc) for this forecasting project.

Note on IFS: 20240305 appears to be the first day that IFS has the parameters we need, so queries will start on the following Sunday (202240310)

In [2]:
import pandas as pd
import glob
import os
import datetime
# import pvlib
from pv_model import model_pv_power
from forecast_solar import get_solar_forecast_fast, get_solar_forecast_ensemble, get_solar_forecast_ensemble_subset
import matplotlib.pyplot as plt
# import scipy
import numpy as np

Read in specs that were processed and aggregated in the last notebook

In [3]:
folder = 'output_specs' 
filename = 'agg_specs_orig_df.pkl'
file_to_read = os.path.join(folder, filename)
agg_specs_orig_df = pd.read_pickle(file_to_read)
agg_specs_orig_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
latitude,32.811041,31.719983,33.021601,31.888,31.255,33.404486,30.4214,28.883196,31.435932,31.030996,32.104125,33.467803,32.4744,32.460054,32.533273,34.380532,26.090103,33.261505,29.271667,29.242304
longitude,-99.91749,-104.442292,-99.61463,-100.825,-102.272,-96.082472,-97.4614,-99.178605,-99.796613,-102.488209,-100.162375,-95.370856,-95.7156,-102.672809,-96.428985,-100.099625,-97.798453,-97.238057,-98.444722,-95.658144
nameplate_dc,131.5944,256.362179,94.507156,130.434783,234.0,170.854369,186.0,176.0,319.0,172.362353,143.08,260.0,82.098305,133.931,182.009816,165.306333,184.861111,191.275636,53.936118,162.4
nameplate_ac,100.8,188.2,74.9,100.0,180.0,125.7,144.0,132.4,250.0,126.3,102.2,198.5,59.8,100.7,147.6,121.4,137.5,148.8,39.2,120.0
mount_type,single-axis,single-axis,single-axis,single-axis,single-axis,single-axis,single-axis,single-axis,single-axis,single-axis,single-axis,single-axis,single-axis,single-axis,single-axis,single-axis,single-axis,single-axis,single-axis,single-axis


In [4]:
num_plants = len(agg_specs_orig_df)
num_plants

20

Now, let's make *ensemble* forecasts.

The full range will use:

```python
dates = pd.date_range(start = '2024-03-10 12:00', end = '2025-03-02 12:00', freq = '7D')
```

but we can start with a shorter range.

In [5]:
latitudes = agg_specs_orig_df.loc['latitude'].to_list()
longitudes = agg_specs_orig_df.loc['longitude'].to_list()

dates = pd.date_range(start = '2024-03-10 12:00', end = '2025-03-02 12:00', freq = '7D')

# dates = pd.date_range(start = '2024-03-10 12:00', end = '2024-03-11 12:00', freq = '7D')
run_length = 168
# run_length = 18
lead_time_to_start = 24
# num_members = 10
attempts = 20

In [6]:
dfs = []
for date in dates:
    resource_data_temp = get_solar_forecast_ensemble(
        latitude=latitudes,
        longitude=longitudes,
        init_date=date,
        run_length=run_length,
        lead_time_to_start=lead_time_to_start,
        model='ifs',
        attempts=attempts,
        # num_members=num_members,
    )
    # add init_time as a column, then convert to index
    resource_data_temp['init_time'] = date
    resource_data_temp.set_index(['init_time', resource_data_temp.index], inplace=True)
    dfs.append(resource_data_temp)
    print('done with init_time: ' + str(date))

resource_data_ens = pd.concat(dfs)
resource_data_ens['albedo'] = 0.2 # fill in albedo data

done with init_time: 2024-03-10 12:00:00
done with init_time: 2024-03-17 12:00:00


Exception has occured : ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Exception has occured : ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))


done with init_time: 2024-03-24 12:00:00
done with init_time: 2024-03-31 12:00:00
done with init_time: 2024-04-07 12:00:00
done with init_time: 2024-04-14 12:00:00
done with init_time: 2024-04-21 12:00:00
done with init_time: 2024-04-28 12:00:00


Exception has occured : ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))


done with init_time: 2024-05-05 12:00:00
done with init_time: 2024-05-12 12:00:00


...

done with init_time: 2025-02-02 12:00:00
done with init_time: 2025-02-09 12:00:00


Exception has occured : ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Exception has occured : ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))


done with init_time: 2025-02-16 12:00:00


Exception has occured : ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Exception has occured : ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Exception has occured : ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))


done with init_time: 2025-02-23 12:00:00
done with init_time: 2025-03-02 12:00:00


In [7]:
resource_data_ens = pd.concat(dfs)
resource_data_ens['albedo'] = 0.2 # fill in albedo data

In [8]:
resource_data_ens[resource_data_ens['point']==0].head()

ghi_csi         ghi  \
init_time           valid_time                                        
2024-03-10 12:00:00 2024-03-11 12:30:00+00:00  0.928330    0.000000   
                    2024-03-11 13:30:00+00:00  0.928330   72.504496   
                    2024-03-11 14:30:00+00:00  0.928330  276.400493   
                    2024-03-11 15:30:00+00:00  1.016201  506.267942   
                    2024-03-11 16:30:00+00:00  1.016201  670.212579   

                                                      dni         dhi  \
init_time           valid_time                                          
2024-03-10 12:00:00 2024-03-11 12:30:00+00:00    0.000000    0.000000   
                    2024-03-11 13:30:00+00:00  156.945061   54.387953   
                    2024-03-11 14:30:00+00:00  506.854800  111.988382   
                    2024-03-11 15:30:00+00:00  778.527934  109.962433   
                    2024-03-11 16:30:00+00:00  821.268263  130.759493   

                                                ghi_clear  member  point  \
init_time           valid_time                                             
2024-03-10 12:00:00 2024-03-11 12:30:00+00:00    0.000000       1      0   
                    2024-03-11 13:30:00+00:00   78.102100       1      0   
                    2024-03-11 14:30:00+00:00  297.739588       1      0   
                    2024-03-11 15:30:00+00:00  498.196465       1      0   
                    2024-03-11 16:30:00+00:00  659.527318       1      0   

                                                temp_air  wind_speed  albedo  
init_time           valid_time                                                
2024-03-10 12:00:00 2024-03-11 12:30:00+00:00   7.621001           2     0.2  
                    2024-03-11 13:30:00+00:00   9.051828           2     0.2  
                    2024-03-11 14:30:00+00:00  10.482655           2     0.2  
                    2024-03-11 15:30:00+00:00  12.317902           2     0.2  
                    2024-03-11 16:30:00+00:00  14.571279           2     0.2

In [9]:
start = resource_data_ens.reset_index()['init_time'].iloc[0].strftime('%Y%m%d')
end = resource_data_ens.reset_index()['init_time'].iloc[-1].strftime('%Y%m%d')

resource_data_ens.to_pickle('forecasts/resource_data_ens_' + start + '-' + end + '.pkl')
resource_data_ens.to_csv('forecasts/resource_data_ens_' + start + '-' + end + '.csv')

Run this if needed to read the CSV back in:

In [1]:
# resource_data_ens = pd.read_csv('forecasts/resource_data_ens.csv', parse_dates=['valid_time'], index_col='valid_time')

Now, let's model power using the resource forecasts to get power forecasts:

In [ ]:
# empty lists
ens_temp_list = []

for member in range(1, num_members+1):
    for plant_number in range(num_plants):
        # pull plant data
        plant_data = agg_specs_orig_df[plant_number].to_dict()

        # get the resource forecast for the plant
        resource_fcast_ens = resource_data_ens[(resource_data_ens['point']==plant_number) \
            & (resource_data_ens['member']==member)]
        
        resource_fcast_ens = resource_fcast_ens.reset_index().set_index('valid_time')

        # model power
        power_fcasts_ens, _ = model_pv_power(resource_fcast_ens, **plant_data)

        power_fcasts_ens.name = 'power'

        # convert to dataframe and add 'point' (plant number)
        power_fcasts_ens = power_fcasts_ens.to_frame()
        power_fcasts_ens['point'] = plant_number
        power_fcasts_ens['member'] = member

        ens_temp_list.append(power_fcasts_ens)

power_fcasts_ens_all = pd.concat(ens_temp_list)

In [ ]:
for member in range(1, num_members+1):
    for plant_number in range(num_plants):
        power_fcasts_ens_all[(power_fcasts_ens_all['member'] == member) & (power_fcasts_ens_all['point'] == plant_number)]['power'].plot(label=('ens mem ' + str(member)), drawstyle='steps-mid')

And write resulting dataframes to CSV and pickle files.

In [ ]:
# power_fcasts_ens_all.to_pickle('forecasts/power_fcasts_ens_all.pkl')
# power_fcasts_ens_all.to_pickle('forecasts/power_fcasts_ens_all.pkl')